In [8]:
from google.cloud import storage

In [59]:
import os

In [6]:
#Importer les bibliothèques nécessaires (pandas et pyspark) dans le script pyspark
import pandas as pd

In [7]:
import openpyxl

ModuleNotFoundError: No module named 'openpyxl'

In [ ]:
from pyspark.sql.functions import lit

In [ ]:
import pyspark.sql
from pyspark.sql import *

In [ ]:
import requests

In [ ]:
from pyspark import SparkFiles

In [ ]:
import zipfile
import io
import os

In [ ]:
#Création de la session spark 
spark = SparkSession.builder.appName("Excel to DataFrame").getOrCreate() 

In [ ]:
def TelechargementData():

    url = "https://services.eaufrance.fr/documents/openData/SISPEA_FR_"+str(annee)+"_AEP.zip"

    reponse = requests.get(url)

    content = reponse.content

    # Création d'un objet ZipFile à partir du contenu ZIP téléchargé
    zip_file = zipfile.ZipFile(io.BytesIO(content))

    # Création du répertoire de destination s'il n'existe pas
    os.makedirs("Data", exist_ok=True)

    # Extraction des fichiers du fichier ZIP et enregistrement dans le répertoire de destination
    zip_file.extractall("Data")
    
    # Fermeture du fichier ZIP
    zip_file.close()

In [ ]:
def lecturePandas(annee):
    ''' Cette fonction permet de lire un fichier excel et de le convertir en dataFrame Pandas'''

    #Pour lire le fichier excel on utilise Pandas 
    #On donne le chemin du fichier excel
    excel_file_path = "./Data/SISPEA_FR_"+str(annee)+"_AEP.xls"
    #On donne le nom de l'onglet
    sheet_name = "Entités de gestion"
    #Lecture du fichier excel avec pandas 
    pandas_df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

    return pandas_df

Je supprime la colonne fin de contrat, car elle creer un conflit qui ne me permet pas de convertir mon dataframe en table spark.

In [ ]:
def supprimerLaColonneDateFin(pandas_df):
    '''Cette fonction supprime la colonne dateFinDeContrat dans mon dataframe Pandas'''
    pandas_df.drop(["Date de fin de contrat"], axis=1, inplace=True)

    return pandas_df

In [ ]:
def convertirPandasEnSpark(pandas_df):
    '''Cette fonction converti la table pandas en table spark'''
    #Conversion du DataFrame Pandas en DataFrame Spark
    table = spark.createDataFrame(pandas_df)
    
    return table

In [ ]:
def choixColonnes(table_spark):
    ''' Cette fonction selectionne les colonnes N° SIREN,VP.224,VP.225,VP.226,VP.227,VP.228,VP.229,VP.231,VP.232,VP.234 '''
    
    table_bonne_colonne=table_spark.select("N° SIREN","`VP.224`","`VP.225`", "`VP.226`", "`VP.227`","`VP.228`","`VP.229`","`VP.231`", "`VP.232`","`VP.234`")

    return table_bonne_colonne

Je renomme les colonnes de mon tableau

In [ ]:
def renommageColonnes(table_spark):
    ''' Cette fonction renomme les colonnes de la table '''

    table_renamed = table_spark.withColumnRenamed("VP.224", "Indice linéaire de consommation")\
        .withColumnRenamed("VP.225", "Rendement sur les 3 années précédentes")\
        .withColumnRenamed("VP.226", "Rendement seuil par défaut")\
        .withColumnRenamed("VP.227", "Rendement seuil en ZRE")\
        .withColumnRenamed("VP.228", "Densité linéaire d'abonnés")\
        .withColumnRenamed("VP.229", "Ratio habitants par abonnés")\
        .withColumnRenamed("VP.231", "Consommation moyenne par abonné")\
        .withColumnRenamed("VP.232", "Volumes consommés comptabilisés")\
        .withColumnRenamed("VP.234", "Volume produit + Volume importé")
    return table_renamed

In [ ]:
def ajoutcol_annee(table_spark,annee):
    # Add new constanst column
    dataframe_colannee = table_spark.withColumn("Annee", lit(annee))
    return dataframe_colannee

Je sauvegarde mon tableau dans un fichier csv

In [ ]:
def transformationCSV(table_spark):
    ''' Cette fonction enregistre une table spark au format CSV '''
    table_spark.write.csv("./Local Disk/Data/consommation_eau.csv", header=True, mode="append")

In [ ]:
def main(annee):
    
    TelechargementData()
    
    pandas_dataframe = lecturePandas(annee)

    pandas_dataframe_propre = supprimerLaColonneDateFin(pandas_dataframe)

    spark_dataframe = convertirPandasEnSpark(pandas_dataframe_propre)

    df_bonnes_colonnes = choixColonnes(spark_dataframe)

    df_bons_noms = renommageColonnes(df_bonnes_colonnes)
    
    dataframe_colannee = ajoutcol_annee(df_bons_noms, annee)

    transformationCSV(dataframe_colannee)
    

In [ ]:
# Selection des années à traiter.
annee_min = 2008
anne_max = 2021

In [ ]:
#Execution du programme

for annee in range(annee_min,anne_max+1):
    main(annee)

In [51]:
## lien URI du bucket crée 
DATALAKE_PATH = "gs://code_de_source_lake"


In [52]:
## variables pour envoyer mon fichier csv sur mon bucket

BUCKET_NAME = "code_de_source_lake"
local_folder_path = "./Data/Output_data"
gcs_file_path = "consommation_eau.csv"



In [53]:
# Nom du dossier dans le bucket GCS
gcs_folder_name = 'Output_data_consommationeau'

In [54]:
# Initialiser le client de stockage GCP
client = storage.Client()

In [55]:
# Récupérer le bucket
bucket = client.bucket(BUCKET_NAME)


In [56]:
# Envoyer le fichier dans le bucket
blob = bucket.blob(gcs_file_path)

In [60]:
# Parcourir les fichiers du dossier local
for root, dirs, files in os.walk(local_folder_path):
    for file in files:
        local_file_path = os.path.join(root, file)
        gcs_file_path = os.path.join(gcs_folder_name, os.path.relpath(local_file_path, local_folder_path))
        
        # Créer un objet Blob dans le bucket
        blob = bucket.blob(gcs_file_path)
        
        # Charger le contenu du fichier local dans le Blob
        blob.upload_from_filename(local_file_path)

        print(f'Fichier {local_file_path} envoyé vers {gcs_file_path}.')

Fichier ./Data/Output_data/part-00006-99a5bfbe-0173-421b-a4e6-e6674b63283a-c000.csv envoyé vers Output_data_consommationeau/part-00006-99a5bfbe-0173-421b-a4e6-e6674b63283a-c000.csv.
Fichier ./Data/Output_data/part-00004-c79c1aaa-31e0-4991-8398-ad7cf9f652b9-c000.csv envoyé vers Output_data_consommationeau/part-00004-c79c1aaa-31e0-4991-8398-ad7cf9f652b9-c000.csv.
Fichier ./Data/Output_data/part-00009-c79c1aaa-31e0-4991-8398-ad7cf9f652b9-c000.csv envoyé vers Output_data_consommationeau/part-00009-c79c1aaa-31e0-4991-8398-ad7cf9f652b9-c000.csv.
Fichier ./Data/Output_data/part-00003-c79c1aaa-31e0-4991-8398-ad7cf9f652b9-c000.csv envoyé vers Output_data_consommationeau/part-00003-c79c1aaa-31e0-4991-8398-ad7cf9f652b9-c000.csv.
Fichier ./Data/Output_data/part-00001-c79c1aaa-31e0-4991-8398-ad7cf9f652b9-c000.csv envoyé vers Output_data_consommationeau/part-00001-c79c1aaa-31e0-4991-8398-ad7cf9f652b9-c000.csv.
Fichier ./Data/Output_data/part-00008-c79c1aaa-31e0-4991-8398-ad7cf9f652b9-c000.csv envoyé

In [58]:
# J'ouvre mon fichier pour le lire en binaire
#with open (local_file_path, "rb") as fichier:
#J'upload le contenu de mon fichier dans mon blob
    #blob.upload_from_file(fichier)

#print(f"Le fichier {local_file_path} a été upload dans le bucket {BUCKET_NAME}.")